# Feature extraction via KimiaNet

Link to the GitHub repository: https://github.com/KimiaLabMayo/KimiaNet/blob/main/KimiaNet_Feature_Extraction_Code_Samples/KimiaNet_Keras_Feature_Extraction.py



**Before running this jupyter notebook and all the other feature extraction models, pass through the `0_gpu_processing_activation.ipynb` notebook to set Tensorflow to run on the GPUs.**


---

# 0.- Setting things up

When importing all these packages for the first time (only for the first time your run this cell in the current Jupyter session), you'll get some warning messages. Ignore them as they are related to the CPU and are also the product of a bug and compatibility with `Keras` package.

Setting the environment to work properly is not easy at all. In the main features extraction folder you can find the requirements for the used environment: `example_kimianet_11_8`.\
Despite the unusual name, it works fine and it created in the following way:\
```sh
conda create -n example_kimianet_11_8 python=3.9 cudnn=9.3.0 cudatoolkit=11.8 -c nvidia -c conda-forge
```
You can change the `cudnn` version to a most recent one, but the most import thing is to have it at least in `9.3.0` as the KimiaNet NN architecture was build in this version.\


Once you activate the environment, the first installation that you have to do is the Tensorflow one, thanks to this command:\
```sh
python3 -m pip install 'tensorflow[and-cuda]'
```

In [1]:
import os
import tensorflow as tf
import subprocess

2025-02-20 15:33:27.037929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740062007.056541  509532 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740062007.062280  509532 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-20 15:33:27.081329: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

import regex
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob, pickle, skimage.io, pathlib
import numpy as np
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, models, Model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Lambda
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

In [3]:
os.environ['XLA_FLAGS'] = '--xla_gpu_strict_conv_algorithm_picker=false'
os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=0"

Checking if the first chosen GPU was, in fact, selected.

In [4]:
# List physical devices available (CPU, GPU)
gpus = tf.config.list_physical_devices('GPU')
print("Available GPUs:", gpus)

for gpu in gpus:
    print("Device Name:", gpu.name)
    print("Device Type:", gpu.device_type)


Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Device Name: /physical_device:GPU:0
Device Type: GPU
Device Name: /physical_device:GPU:1
Device Type: GPU


In [5]:
if gpus:
    # Set only GPU:1 to be visible to TensorFlow
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')  # Adjusting to GPU 1. It maybe be set on 0 depending on the visible GPU that we've selected
        tf.config.experimental.set_memory_growth(gpus[0], True)  # Enable memory growth
    except RuntimeError as e:
        print(f"Error setting GPU configuration: {e}")

# Verify that only GPU:1 is used
print("Configured devices:", tf.config.get_visible_devices())


Configured devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 1.- Configuring variables

## 1.1 - Setting the working path

In [6]:
os.chdir("/disk2/user/gabgam/work/gigi_env/the_project/3_features_extraction/")
print(os.getcwd())

/disk2/work/gabgam/gigi_env/the_project/3_features_extraction


Model name.

In [7]:
model = "kimianet"

Model weights download link:
https://github.com/KimiaLabMayo/KimiaNet/blob/main/KimiaNet_Weights/weights/KimiaNetKerasWeights.h5

Or, with code:

In [8]:
import requests

# Define the URL of the raw file
url = "https://github.com/KimiaLabMayo/KimiaNet/raw/main/KimiaNet_Weights/weights/KimiaNetKerasWeights.h5"
# Define the file path where it will be saved
file_path = "./models/kimianet/KimiaNetKerasWeights.h5"

# Send a GET request to download the file
response = requests.get(url, stream=True)

# Save the file to the specified path
if response.status_code == 200:
    with open(file_path, "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)
    print(f"Downloaded successfully to {file_path}")
else:
    print(f"Failed to download. Status code: {response.status_code}")


Downloaded successfully to ./models/kimianet/KimiaNetKerasWeights.h5


Importing the summary files for both the techniques and the tiles dimension.\
These contain the path to all the tiles folder, the original and the normalised ones.

In [9]:
SUMMARY_SATAC_100um = "/disk2/work/gabgam/gigi_env/the_project/2_image_normalisation/output/satac_C1/v3_allspots/tiles_100/final_summary_for_all_100um_normalised_tiles.csv"
SUMMARY_SATAC_68um = "/disk2/work/gabgam/gigi_env/the_project/2_image_normalisation/output/satac_C1/v3_allspots/tiles_68/final_summary_for_all_68um_normalised_tiles.csv"

SUMMARY_VISIUM_100um = "/disk2/work/gabgam/gigi_env/the_project/2_image_normalisation/output/visium_2022_FF_WG_10X/img_not_changed_allspots/tiles_100/final_summary_for_all_100um_normalised_tiles.csv"
SUMMARY_VISIUM_68um = "/disk2/work/gabgam/gigi_env/the_project/2_image_normalisation/output/visium_2022_FF_WG_10X/img_not_changed_allspots/tiles_68/final_summary_for_all_68um_normalised_tiles.csv"

Extracting sample name and coordinates systems from files name.

In [10]:
# for sATAC
temp_satac_names = SUMMARY_SATAC_100um.split('output/')[1].split('/tiles')[0].split("/")

SAMPLE_SATAC = temp_satac_names[0]
IMAGE_VERSION_AND_COORDINATE_SYSTEM_SATAC = temp_satac_names[1]

# for Visium
temp_visium_names = SUMMARY_VISIUM_100um.split('output/')[1].split('/tiles')[0].split("/")

SAMPLE_VISIUM = temp_visium_names[0]
IMAGE_VERSION_AND_COORDINATE_SYSTEM_VISIUM = temp_visium_names[1]


print(f"Processing for:\nsATAC sample: {SAMPLE_SATAC}\nsATAC coordinates system: {IMAGE_VERSION_AND_COORDINATE_SYSTEM_SATAC}\n\nVisium sample: {SAMPLE_VISIUM}\nVisium coordinates system: {IMAGE_VERSION_AND_COORDINATE_SYSTEM_VISIUM}")

Processing for:
sATAC sample: satac_C1
sATAC coordinates system: v3_allspots

Visium sample: visium_2022_FF_WG_10X
Visium coordinates system: img_not_changed_allspots


Choosing the reference and the name system based on `TARGET_IS_<filename>`.

In [11]:
# path to the target image
PATH_TO_REFERENCE = "../2_image_normalisation/reference_images/reference_full.jpeg"
# `TARGET_IS_<filename>`
TARGET_IS = "target_is_reference_full"

Creating a dictionary for the path to the file, but with the reference image already defined.

In [12]:
complete_dict_summaries = {f"{SAMPLE_SATAC}_&_{IMAGE_VERSION_AND_COORDINATE_SYSTEM_SATAC}_&_{TARGET_IS}_100um": SUMMARY_SATAC_100um,
                  f"{SAMPLE_SATAC}_&_{IMAGE_VERSION_AND_COORDINATE_SYSTEM_SATAC}_&_{TARGET_IS}_68um": SUMMARY_SATAC_68um,
                  f"{SAMPLE_VISIUM}_&_{IMAGE_VERSION_AND_COORDINATE_SYSTEM_VISIUM}_&_{TARGET_IS}_100um": SUMMARY_VISIUM_100um,
                  f"{SAMPLE_VISIUM}_&_{IMAGE_VERSION_AND_COORDINATE_SYSTEM_VISIUM}_&_{TARGET_IS}_68um": SUMMARY_VISIUM_68um}
complete_dict_summaries = {name: pd.read_csv(path, index_col = [0]) for name, path in complete_dict_summaries.items()}
complete_dict_summaries

{'satac_C1_&_v3_allspots_&_target_is_reference_full_100um':                        Normalisation Method                    Target  \
 0                              ORIGINAL WSI                         -   
 1     fromWSI_histomicsTK_macenko_nomasking  target_is_reference_full   
 2   fromWSI_histomicsTK_macenko_withmasking  target_is_reference_full   
 3               fromWSI_staintools_reinhard  target_is_reference_full   
 0             histomicsTK_macenko_nomasking  target_is_reference_full   
 1           histomicsTK_macenko_withmasking  target_is_reference_full   
 2            histomicsTK_reinhard_nomasking         target_is_default   
 3            histomicsTK_reinhard_nomasking  target_is_reference_full   
 4          histomicsTK_reinhard_withmasking         target_is_default   
 5          histomicsTK_reinhard_withmasking  target_is_reference_full   
 6                          stainGAN_model_A                         -   
 7                          stainGAN_model_B         

Specify the normalisation methods that have been chosen and the `ORIGINAL WSI` with the exact name in the summaries.

In [13]:
SELECTED_METHODS = ["ORIGINAL WSI", "fromWSI_histomicsTK_macenko_nomasking", "histomicsTK_macenko_nomasking", "stainNET"]

Choosing the weights.

In [14]:
NETWORK_WEIGHTS_ADDRESS = f"models/{model}/KimiaNetKerasWeights.h5"

Setting the output path.

In [15]:
comparison_of_analyses = f"{SAMPLE_SATAC}_{IMAGE_VERSION_AND_COORDINATE_SYSTEM_SATAC}_&_{SAMPLE_VISIUM}_{IMAGE_VERSION_AND_COORDINATE_SYSTEM_VISIUM}"
features_output_path = f"output/{model}/{comparison_of_analyses}/"

os.makedirs(features_output_path, exist_ok = True)
features_output_path

'output/kimianet/satac_C1_v3_allspots_&_visium_2022_FF_WG_10X_img_not_changed_allspots/'

# 2. - Preprocessing and Feature Extraction Functions

In [16]:
summary_satac_100um = pd.read_csv(SUMMARY_SATAC_100um, index_col= [0])
summary_satac_100um

,Normalisation Method,Target,Number of tiles,Path to folder
0,ORIGINAL WSI,-,3631,/disk2/work/gabgam/gigi_env/the_project/1_tili...
1,fromWSI_histomicsTK_macenko_nomasking,target_is_reference_full,3631,/disk2/work/gabgam/gigi_env/the_project/1_tili...
2,fromWSI_histomicsTK_macenko_withmasking,target_is_reference_full,3631,/disk2/work/gabgam/gigi_env/the_project/1_tili...
3,fromWSI_staintools_reinhard,target_is_reference_full,3631,/disk2/work/gabgam/gigi_env/the_project/1_tili...
0,histomicsTK_macenko_nomasking,target_is_reference_full,3631,/disk2/work/gabgam/gigi_env/the_project/2_imag...
1,histomicsTK_macenko_withmasking,target_is_reference_full,3631,/disk2/work/gabgam/gigi_env/the_project/2_imag...
2,histomicsTK_reinhard_nomasking,target_is_default,3631,/disk2/work/gabgam/gigi_env/the_project/2_imag...
3,histomicsTK_reinhard_nomasking,target_is_reference_full,3631,/disk2/work/gabgam/gigi_env/the_project/2_imag...
4,histomicsTK_reinhard_withmasking,target_is_default,3631,/disk2/work/gabgam/gigi_env/the_project/2_imag...
5,histomicsTK_reinhard_withmasking,target_is_reference_full,3631,/disk2/work/gabgam/gigi_env/the_project/2_imag...


Let's import the summary files with the paths to the folders and select the paths to the tiles folders for the ORIGINAL WSI and the methods that worked out.

In [17]:
filter_summaries_by_method = {name: df[df['Normalisation Method'].isin(SELECTED_METHODS)] for name, df in complete_dict_summaries.items()}

Selecting by the specified normalisation methods and the target.

In [18]:
filter_summaries_by_method = {name: df[df['Normalisation Method'].isin(SELECTED_METHODS) & df["Target"].isin(["-", TARGET_IS])].reset_index() for name, df in complete_dict_summaries.items()}
filter_summaries_by_method

{'satac_C1_&_v3_allspots_&_target_is_reference_full_100um':    index                   Normalisation Method                    Target  \
 0      0                           ORIGINAL WSI                         -   
 1      1  fromWSI_histomicsTK_macenko_nomasking  target_is_reference_full   
 2      0          histomicsTK_macenko_nomasking  target_is_reference_full   
 3      8                               stainNET                         -   
 
    Number of tiles                                     Path to folder  
 0             3631  /disk2/work/gabgam/gigi_env/the_project/1_tili...  
 1             3631  /disk2/work/gabgam/gigi_env/the_project/1_tili...  
 2             3631  /disk2/work/gabgam/gigi_env/the_project/2_imag...  
 3             3631  /disk2/work/gabgam/gigi_env/the_project/2_imag...  ,
 'satac_C1_&_v3_allspots_&_target_is_reference_full_68um':    index                   Normalisation Method                    Target  \
 0      0                           ORIGINAL WS

These are the KimiaNet features extraction functions.\
I've modified the `preprocessing_fn` one to make sure that it adapts itself to the size of the input image, and I don't have to specify it.

In [19]:
# def preprocessing_fn(input_batch, network_input_patch_width=None):
#     org_input_size = tf.shape(input_batch)[1:3]  # Extract height & width dynamically
#     network_input_patch_width = tf.reduce_max(org_input_size)  # Use max dimension as base

#     scaled_input_batch = tf.cast(input_batch, 'float32') / 255.0  # Ensuring dtype is float32
#     # resized_input_batch = tf.image.resize(scaled_input_batch, 
#     #                                       (network_input_patch_width, network_input_patch_width))
    
#     resized_input_batch = tf.cond(tf.equal(org_input_size, network_input_patch_width),
#                                   lambda: scaled_input_batch, 
#                                   lambda: tf.image.resize(scaled_input_batch, 
#                                                           (network_input_patch_width, network_input_patch_width)))
    
#     mean = [0.485, 0.456, 0.406]
#     std = [0.229, 0.224, 0.225]
#     data_format = "channels_last"
#     mean_tensor = tf.constant(-np.array(mean), dtype=tf.float32)  # Ensuring mean tensor is float32
#     standardized_input_batch = tf.keras.backend.bias_add(resized_input_batch, mean_tensor, data_format)
#     standardized_input_batch /= std
#     return standardized_input_batch



# # ------------------------------------------------------------------------------------------------------
# def kimianet_feature_extractor(weights_address):
#     if not os.path.exists(weights_address):
#         raise FileNotFoundError(f"Weights file not found at {weights_address}")

#     # Allow input of arbitrary sizes
#     dnx = DenseNet121(include_top=False, weights=weights_address, input_shape=(None, None, 3), pooling='avg')

#     kn_feature_extractor = Model(inputs=dnx.input, outputs=GlobalAveragePooling2D()(dnx.layers[-3].output))
#     kn_feature_extractor_seq = Sequential([Lambda(preprocessing_fn, input_shape=(None, None, 3), dtype=tf.uint8)])
#     kn_feature_extractor_seq.add(kn_feature_extractor)
    
#     return kn_feature_extractor_seq



# # ------------------------------------------------------------------------------------------------------
# def extract_features(row, network_weights_address, batch_size, img_format):
#     patch_dir = row["Path to folder"]
#     feature_extractor = kimianet_feature_extractor(network_weights_address)
#     patch_adr_list = [pathlib.Path(x) for x in glob.glob(os.path.join(patch_dir, f'*.{img_format}'))]

#     # Debug: Print number of files found
#     print(f"Number of files found: {len(patch_adr_list)}")
    
#     if len(patch_adr_list) == 0:
#         print("No files found. Please check the patch directory and image format.")
#         return

#     feature_dict = {}

#     for batch_st_ind in tqdm(range(0, len(patch_adr_list), batch_size)):
#         batch_end_ind = min(batch_st_ind + batch_size, len(patch_adr_list))
#         batch_patch_adr_list = patch_adr_list[batch_st_ind:batch_end_ind]

#         # Debug: Print current batch size
#         print(f"Processing batch from index {batch_st_ind} to {batch_end_ind}")

#         patch_batch = np.array([skimage.io.imread(str(x)) for x in batch_patch_adr_list])

#         # Determine image size dynamically
#         batch_features = feature_extractor.predict(patch_batch)
#         feature_dict.update(dict(zip([x.stem for x in batch_patch_adr_list], list(batch_features))))

#     print("Feature extraction completed.")
#     return pd.DataFrame.from_dict(feature_dict)

In [24]:
def preprocessing_fn(input_batch, network_input_patch_width):
    org_input_size = tf.shape(input_batch)[1]
    scaled_input_batch = tf.cast(input_batch, 'float32') / 255.0  # Ensuring dtype is float32
    resized_input_batch = tf.cond(tf.equal(org_input_size, network_input_patch_width),
                                  lambda: scaled_input_batch, 
                                  lambda: tf.image.resize(scaled_input_batch, 
                                                          (network_input_patch_width, network_input_patch_width)))
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    data_format = "channels_last"
    mean_tensor = tf.constant(-np.array(mean), dtype=tf.float32)  # Ensuring mean tensor is float32
    standardized_input_batch = tf.keras.backend.bias_add(resized_input_batch, mean_tensor, data_format)
    standardized_input_batch /= std
    return standardized_input_batch


# ------------------------------------------------------------------------------------------------------
def kimianet_feature_extractor(network_input_patch_width, weights_address):
    if not os.path.exists(weights_address):
        raise FileNotFoundError(f"Weights file not found at {weights_address}")
    
    dnx = DenseNet121(include_top=False, weights=weights_address, 
                      input_shape=(network_input_patch_width, network_input_patch_width, 3), pooling='avg')
    kn_feature_extractor = Model(inputs=dnx.input, outputs=GlobalAveragePooling2D()(dnx.layers[-3].output))
    kn_feature_extractor_seq = Sequential([Lambda(preprocessing_fn, 
                                                  arguments={'network_input_patch_width': network_input_patch_width}, 
                                                  input_shape=(None, None, 3), dtype=tf.uint8)])
    kn_feature_extractor_seq.add(kn_feature_extractor)
    return kn_feature_extractor_seq


# ------------------------------------------------------------------------------------------------------
def extract_features(patch_dir, network_weights_address, 
                     network_input_patch_width, batch_size, img_format):
    patch_dir = row["Path to folder"]
    feature_extractor = kimianet_feature_extractor(network_input_patch_width, network_weights_address)
    patch_adr_list = [pathlib.Path(x) for x in glob.glob(os.path.join(patch_dir, f'*.{img_format}'))]
    
    # Debug: Print number of files found
    print(f"Number of files found: {len(patch_adr_list)}")
    
    if len(patch_adr_list) == 0:
        print("No files found. Please check the patch directory and image format.")
        return
    
    feature_dict = {}

    for batch_st_ind in tqdm(range(0, len(patch_adr_list), batch_size)):
        batch_end_ind = min(batch_st_ind + batch_size, len(patch_adr_list))
        batch_patch_adr_list = patch_adr_list[batch_st_ind:batch_end_ind]
        
        # Debug: Print current batch size
        print(f"Processing batch from index {batch_st_ind} to {batch_end_ind}")
        
        patch_batch = np.array([skimage.io.imread(str(x)) for x in batch_patch_adr_list])
        batch_features = feature_extractor.predict(patch_batch)
        feature_dict.update(dict(zip([x.stem for x in batch_patch_adr_list], list(batch_features))))
        
        # with open(extracted_features_save_adr, 'wb') as output_file:
        #     pickle.dump(feature_dict, output_file, pickle.HIGHEST_PROTOCOL)
    
    print("Feature extraction completed.")
    return pd.DataFrame.from_dict(feature_dict)


Let's perform the real features extraction step.

Let's set the batch size (accordingly to GPU computational power), the image file format (tipically `.jpg`).\
The real extraction function will be called inside the dictionary where each key will contain all the info regarding the images in the folder (sample origin, size in micrometers and the eventual normalisation method) and the relative value will be the dataframe originated from the function which will contain all the extracted features per image (images indeces have their own real name).

In the end, all the feature dataframes will be saved as `.pickle` files in the `./output` folder.

In [ ]:
batch_size = 16 #32
img_format = 'jpg'
network_input_patch_width = 1000 #224 #1000


all_extracted_features_from_selected_summaries = {}

# Create a list of (name, row) pairs
entries = [
    (name, row) for name, df in filter_summaries_by_method.items() for _, row in df.iterrows()
]

# Iterate over the list
for name, row in entries:
    key = f"{name}_{row['Normalisation Method']}"  # Using dictionary-like access
    try:
        features = extract_features(row, NETWORK_WEIGHTS_ADDRESS, network_input_patch_width, batch_size, img_format)
        all_extracted_features_from_selected_summaries[key] = features # features as dataframes
    except Exception as e:
        print(f"Error processing {key}: {e}")

# the gpu_timer warning at the end of the output doesn't affect it. It's a bug related to the versions of some packages like nightly and CUDNN:
# https://github.com/tensorflow/tensorflow/issues/71791

# parameters_folder = f"batchsize{batch_size}" if not network_input_patch_width else f"batchsize{batch_size}_inputpathwidth{network_input_patch_width}"  # might be useless

# saving all the dataframes as .pickles files with the specific name from the folder and the normalisation method that was used.
for name, features_df in all_extracted_features_from_selected_summaries.items():
    if network_input_patch_width:
        with open(os.path.join(features_output_path, f"{name}_width{network_input_patch_width}.pickle"), 'wb') as output_file:
            pickle.dump(features_df, output_file, pickle.HIGHEST_PROTOCOL)

    else:
        with open(os.path.join(features_output_path, f"{name}.pickle"), 'wb') as output_file:
            pickle.dump(features_df, output_file, pickle.HIGHEST_PROTOCOL)

# the output is very long, I prefer to clear it after each run.

Number of files found: 3631


  0%|          | 0/227 [00:00<?, ?it/s]

Processing batch from index 0 to 16


I0000 00:00:1740062061.376946  509751 service.cc:148] XLA service 0x7f88ac004740 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740062061.386059  509751 service.cc:156]   StreamExecutor device (0): Quadro RTX 6000, Compute Capability 7.5
2025-02-20 15:34:21.869895: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1740062062.967357  509751 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1740062065.394181  509751 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1740062065.546140  509751 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1740062066.820659  509751 gpu_tim

1/1 ━━━━━━━━━━━━━━━━━━━━ 40s 40s/step


...

100%|██████████| 307/307 [02:18<00:00,  2.22it/s]

Feature extraction completed.


Number of files found: 4898


  0%|          | 0/307 [00:00<?, ?it/s]

Processing batch from index 0 to 16


Saving the dataframes as `.pickle` files (this was done in previous cell as well, here is just for comodity and eventual future modifications of the code).

In [22]:
for name, features_df in all_extracted_features_from_selected_summaries.items():
    with open(os.path.join(features_output_path, f"{name}.pickle"), 'wb') as output_file:
        pickle.dump(features_df, output_file, pickle.HIGHEST_PROTOCOL)

Let's visualise one of them.

In [23]:
all_extracted_features_from_selected_summaries["satac_C1_&_v3_allspots_&_target_is_reference_full_68um_fromWSI_histomicsTK_macenko_nomasking"]

KeyError: 'satac_C1_&_v3_allspots_&_target_is_reference_full_68um_fromWSI_histomicsTK_macenko_nomasking'

---
**Saving the environment requirements**

In [ ]:
# Save package versions to a .txt file
with open("example_kimianet_11_8.txt", "w") as f:
    subprocess.run(["conda", "list", "--explicit"], stdout=f)

---

# Final - Ending the working session on the GPU

This is what ChatGPT propose me, but it doesn't work...

In [ ]:
# from tensorflow.keras import backend as K
# #import gc

# # Clear Keras session
# K.clear_session()
# # Force garbage collection
# #gc.collect()


A way more direct approach could be killing all the processes:

```{bash}
pkill -u <user_name>
```

In [ ]:
# from numba import cuda

# device = cuda.get_current_device()
# device
# device.reset()

In [ ]:
# print(device)

<CUDA device 0 'b'Quadro RTX 6000''>


# discarded codes

In [ ]:
# patch_dir = f"../1_tiling/outputs/{sample}/tiling_output/"
# extracted_features_save_adr = f"./models/{model}/output/{sample}/extracted_features_{model}_{sample}.pickle"
# #tissuetypefile_path = "../data/data_for_34C/V10F03-034_C_S7_Wenwen-annotations.csv" # pathologist annotation

# # Extract the directory path only (exclude filename)
# directory_path = os.path.dirname(extracted_features_save_adr)

# # Check if the path exists, if not, create the directories
# for path in [directory_path]:
#     if not os.path.exists(path):
#         # Create the directory and any necessary intermediate directories  
#         os.makedirs(path)
#         print(f"Created directory: {path}")  
#     else:  
#         print(f"Directory already exists: {path}")

In [ ]:
# def preprocessing_fn(input_batch, network_input_patch_width):
#     org_input_size = tf.shape(input_batch)[1]
#     scaled_input_batch = tf.cast(input_batch, 'float32') / 255.0  # Ensuring dtype is float32
#     resized_input_batch = tf.cond(tf.equal(org_input_size, network_input_patch_width),
#                                   lambda: scaled_input_batch, 
#                                   lambda: tf.image.resize(scaled_input_batch, 
#                                                           (network_input_patch_width, network_input_patch_width)))
#     mean = [0.485, 0.456, 0.406]
#     std = [0.229, 0.224, 0.225]
#     data_format = "channels_last"
#     mean_tensor = tf.constant(-np.array(mean), dtype=tf.float32)  # Ensuring mean tensor is float32
#     standardized_input_batch = tf.keras.backend.bias_add(resized_input_batch, mean_tensor, data_format)
#     standardized_input_batch /= std
#     return standardized_input_batch


# # ------------------------------------------------------------------------------------------------------
# def kimianet_feature_extractor(network_input_patch_width, weights_address):
#     if not os.path.exists(weights_address):
#         raise FileNotFoundError(f"Weights file not found at {weights_address}")
    
#     dnx = DenseNet121(include_top=False, weights=weights_address, 
#                       input_shape=(network_input_patch_width, network_input_patch_width, 3), pooling='avg')
#     kn_feature_extractor = Model(inputs=dnx.input, outputs=GlobalAveragePooling2D()(dnx.layers[-3].output))
#     kn_feature_extractor_seq = Sequential([Lambda(preprocessing_fn, 
#                                                   arguments={'network_input_patch_width': network_input_patch_width}, 
#                                                   input_shape=(None, None, 3), dtype=tf.uint8)])
#     kn_feature_extractor_seq.add(kn_feature_extractor)
#     return kn_feature_extractor_seq


# # ------------------------------------------------------------------------------------------------------
# def extract_features(patch_dir, network_weights_address, 
#                      network_input_patch_width, batch_size, img_format):
#     feature_extractor = kimianet_feature_extractor(network_input_patch_width, network_weights_address)
#     patch_adr_list = [pathlib.Path(x) for x in glob.glob(os.path.join(patch_dir, f'*.{img_format}'))]
    
#     # Debug: Print number of files found
#     print(f"Number of files found: {len(patch_adr_list)}")
    
#     if len(patch_adr_list) == 0:
#         print("No files found. Please check the patch directory and image format.")
#         return
    
#     feature_dict = {}

#     for batch_st_ind in tqdm(range(0, len(patch_adr_list), batch_size)):
#         batch_end_ind = min(batch_st_ind + batch_size, len(patch_adr_list))
#         batch_patch_adr_list = patch_adr_list[batch_st_ind:batch_end_ind]
        
#         # Debug: Print current batch size
#         print(f"Processing batch from index {batch_st_ind} to {batch_end_ind}")
        
#         patch_batch = np.array([skimage.io.imread(str(x)) for x in batch_patch_adr_list])
#         batch_features = feature_extractor.predict(patch_batch)
#         feature_dict.update(dict(zip([x.stem for x in batch_patch_adr_list], list(batch_features))))
        
#         # with open(extracted_features_save_adr, 'wb') as output_file:
#         #     pickle.dump(feature_dict, output_file, pickle.HIGHEST_PROTOCOL)
    
#     print("Feature extraction completed.")
#     return pd.DataFrame.from_dict(feature_dict)


**With patch width**

In [ ]:
# network_input_patch_width = 224 #1000
# batch_size = 32 #16
# img_format = 'jpg'
# #use_gpu = True

# all_extracted_features_from_selected_summaries = {}

# # Create a list of (name, row) pairs
# entries = [
#     (name, row) for name, df in filter_summaries_by_method.items() for _, row in df.iterrows()
# ]

# # Iterate over the list
# for name, row in entries:
#     key = f"{name}_{row['Normalisation Method']}"  # Using dictionary-like access
#     try:
#         features = extract_features(row, NETWORK_WEIGHTS_ADDRESS, network_input_patch_width, batch_size, img_format)
#         all_extracted_features_from_selected_summaries[key] = features
#     except Exception as e:
#         print(f"Error processing {key}: {e}")
